# Semantic Search on PDF Documents with qHNSW Index

##### Note: This example requires a KDB.AI endpoint and API key. Sign up for a free [KDB.AI account](https://kdb.ai/get-started).

This example demonstrates how to use KDB.AI to run semantic search on unstructured text documents.

<div class="alert alert-block alert-info">
<b>Tip:</b> This sample uses ‘qHNSW’ , a new vector index choice in KDB.AI. It will support the same API options as the existing ‘HNSW’ index but with the significant difference that the index is stored on-disk and memory-mapped as required. This means data inserts will have negligible memory and cpu footprints. The vector index can grow and be searched as long as there is disk space available. Among other cases, this stands out as a great index for memory contrained situations such as edge devices.
</div>

Semantic search allows users to perform searches based on the meaning or similarity of the data rather than exact matches. It works by converting the query into a vector representation and then finding similar vectors in the database. This way, even if the query and the data in the database are not identical, the system can identify and retrieve the most relevant results based on their semantic meaning.

### Aim
In this tutorial, we'll walk you through the process of performing semantic search on documents, taking PDFs as example, using KDB.AI as the vector store. We will cover the following topics:

0. Setup
1. Load PDF Data
2. KDB.AI Table Creation (qHNSW index)
3. LlamaIndex index & query_engine setup
4. Retrieve Similar Sentences & RAG
5. Delete the KDB.AI Table

---

## 0. Setup

### Install dependencies

In order to successfully run this sample, note the following steps depending on where you are running this notebook:

-***Run Locally / Private Environment:*** The [Setup](https://github.com/KxSystems/kdbai-samples/blob/main/README.md#setup) steps in the repository's `README.md` will guide you on prerequisites and how to run this with Jupyter.


-***Colab / Hosted Environment:*** Open this notebook in Colab and run through the cells.

### Set Environment Variables

In [ ]:
!pip install llama-index llama-index-llms-openai llama-index-embeddings-openai llama-index-readers-file llama-index-vector-stores-kdbai
!pip install kdbai_client pandas


### Import Packages

In [2]:

import os
from getpass import getpass
import re
import os
import shutil
import time
import urllib

import pandas as pd

from llama_index.core import (
    Settings,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.vector_stores.kdbai import KDBAIVectorStore

import kdbai_client as kdbai

OUTDIR = "pdf"
RESET = True

In [3]:
### !!! Only run this cell if you need to download the data into your environment, for example in Colab
### This downloads research paper pdf into your environment
if os.path.exists("./data/research_paper") == False:
  !mkdir ./data
  !wget -P ./data https://raw.githubusercontent.com/KxSystems/kdbai-samples/main/document_search/data/research_paper.pdf

--2024-09-16 19:41:35--  https://raw.githubusercontent.com/KxSystems/kdbai-samples/main/document_search/data/research_paper.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1206037 (1.1M) [application/octet-stream]
Saving to: ‘./data/research_paper.pdf’

research_paper.pdf  100%[===================>]   1.15M  --.-KB/s    in 0.06s   

2024-09-16 19:41:35 (17.9 MB/s) - ‘./data/research_paper.pdf’ saved [1206037/1206037]



In [4]:
# OpenAI API Key: https://platform.openai.com/api
os.environ["OPENAI_API_KEY"] = (
    os.environ["OPENAI_API_KEY"]
    if "OPENAI_API_KEY" in os.environ
    else getpass("OpenAI API Key: ")
)

OpenAI API Key: ··········


In [5]:
# Set up LlamaIndex Parameters

EMBEDDING_MODEL  = "text-embedding-3-small"
GENERATION_MODEL = 'gpt-4o-mini'

llm = OpenAI(model=GENERATION_MODEL)
embed_model = OpenAIEmbedding(model=EMBEDDING_MODEL)

Settings.llm = llm
Settings.embed_model = embed_model

### Configure Console

In [6]:
pd.set_option("display.max_colwidth", 300)

### Define Helper Functions

In [7]:
def show_df(df: pd.DataFrame) -> pd.DataFrame:
    print(df.shape)
    return df.head()

## 1. Load PDF Data

### Read Text From PDF Document

We LlamaIndex SimpleDirectorReader to read in our PDF file.

The PDF we are using is [this research paper](https://arxiv.org/pdf/2308.05801.pdf) presenting information on the formation of Interstellar Objects in the Milky Way.

In [8]:
reader = SimpleDirectoryReader(
    input_dir="data",
)
documents = reader.load_data()

### Define KDB.AI Session

KDB.AI comes in two offerings:

1. [KDB.AI Cloud](https://trykdb.kx.com/kdbai/signup/) - For experimenting with smaller generative AI projects with a vector database in our cloud.
2. [KDB.AI Server](https://trykdb.kx.com/kdbaiserver/signup/) - For evaluating large scale generative AI applications on-premises or on your own cloud provider.

Depending on which you use there will be different setup steps and connection details required.

##### Option 1. KDB.AI Cloud

To use KDB.AI Cloud, you will need two session details - a URL endpoint and an API key.
To get these you can sign up for free [here](https://trykdb.kx.com/kdbai/signup).

You can connect to a KDB.AI Cloud session using `kdbai.Session` and passing the session URL endpoint and API key details from your KDB.AI Cloud portal.

If the environment variables `KDBAI_ENDPOINTS` and `KDBAI_API_KEY` exist on your system containing your KDB.AI Cloud portal details, these variables will automatically be used to connect.
If these do not exist, it will prompt you to enter your KDB.AI Cloud portal session URL endpoint and API key details.

In [ ]:
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else getpass("KDB.AI API key: ")
)

In [11]:
session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

##### Option 2. KDB.AI Server

To use KDB.AI Server, you will need download and run your own container.
To do this, you will first need to sign up for free [here](https://trykdb.kx.com/kdbaiserver/signup/).

You will receive an email with the required license file and bearer token needed to download your instance.
Follow instructions in the signup email to get your session up and running.

Once the [setup steps](https://code.kx.com/kdbai/gettingStarted/kdb-ai-server-setup.html) are complete you can then connect to your KDB.AI Server session using `kdbai.Session` and passing your local endpoint.

In [ ]:
# session = kdbai.Session(endpoint="http://localhost:8082")

### Define Vector DB Table Schema

The next step is to define a schema for our KDB.AI table where we will store our embeddings. Our table will have two columns.

At this point you will select the index and metric you want to use for searching.

In this case, we will use the qHNSW index, Euclidean Distance (L2) for the search metric, and we specify the number of dimensions of our embeddings (384).

In [12]:
pdf_schema = dict(
    columns=[
        dict(name="document_id", pytype="bytes"),
        dict(name="text", pytype="bytes"),
        dict(
            name="embedding",
            vectorIndex=dict(type="qHnsw", metric="L2", dims=1536),
        ),
    ]
)

### Create Vector DB Table

Use the KDB.AI `create_table` function to create a table that matches the defined schema in the vector database.

In [13]:
# First ensure the table does not already exist
try:
    session.table("pdf").drop()
except kdbai.KDBAIException:
    pass

In [14]:
table = session.create_table("pdf", pdf_schema)

We can use `query` to see our table exists but is empty.

In [15]:
table.query()

,document_id,text,embedding


## 3. LlamaIndex index & query_engine setup
Define the index: using KDB.AI as the vector store, chunk, embed, and load the document into KDB.AI

In [16]:
vector_store = KDBAIVectorStore(table)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    transformations=[SentenceSplitter(chunk_size=2048, chunk_overlap=0)],
)

### Verify Data Has Been Inserted

Running `table.query()` should show us that data has been added.

In [18]:
show_df(table.query())

(22, 3)


,document_id,text,embedding
0,b'67d676f7-d19e-4976-b3ec-9ed56945f58d',"b'Draft version August 14, 2023\nTypeset using L ATEX default style in AASTeX631\nThe Galactic Interstellar Object Population: A Framework for Prediction and Inference\nMatthew J. Hopkins\n ,1Chris Lintott\n ,1Michele T. Bannister\n ,2J. Ted Mackereth\n ,3, 4, 5, \xe2\x88\x97and\nJohn C. Forbes\...","[-0.0023145927, 0.044429615, 0.014580944, -0.00033039076, 0.01776392, -0.02068275, 0.023720447, -0.016746953, -0.0045235264, 0.00025836928, -0.0173677, 0.0032902877, -0.005358893, -0.026467582, 0.019216733, 0.03373164, -0.020075211, -0.0029782634, -0.021329913, 0.013260206, 0.0025556274, -0.0153..."
1,b'4a96618f-6cd6-4acf-9185-13186a9d6161',"b'2 Hopkins et al.\nInitially it was expected that interstellar objects would display cometary characteristics (e.g. Jewitt 2003). The pop-\nulation\xe2\x80\x99s dominant dynamical formation mechanisms would preferentially harvest more distant, ice-rich planetesimals\nfrom the disks of the sourc...","[0.0053878566, 0.00944298, -0.0030522258, 0.0056825313, 0.01017297, -0.025194675, 0.032307047, -0.013682276, 0.01757332, 0.024082948, -0.0066536176, 0.008003093, -0.012054868, -0.039191715, -0.014305111, 0.017975148, -0.033458956, -0.0035360947, -0.02976213, -0.0025147796, 0.01630086, -0.0131130..."
2,b'c7a20123-9eda-4d90-90a5-ecc256584c4e',"b'The Galactic ISO Population 3\nprocesses modelled, and demonstrate this method by constraining the metallicity dependence of the ISO production\nrate.\n2.APOGEE AND STELLAR DENSITY MODELLING\nTo predict the distribution of ISOs in the Milky Way, we first obtain the distribution of all stars th...","[0.03247003, -0.011693971, 0.067545675, 0.0043938514, 0.008624852, -0.021696795, 0.017212123, -0.01583415, -0.021446254, 0.0010279984, -0.019642364, 0.015596136, -0.035050593, 0.0051987125, 0.011982093, 0.017274758, -0.0075412644, -0.010936086, -0.031943895, 0.022072604, 0.009188567, 0.015470867..."
3,b'e8f947ca-f209-44ce-841d-81342791d553',"b'4 Hopkins et al.\nfollows that the probability of finding a point (i.e. an observed star) with observables in the infinitesimal volume \xce\xb4O\nis given by \xce\xbb(O)\xce\xb4O, and the total number of points (i.e. stars observed) is a Poisson random variable with mean and\nvariance \xce\x9b...","[0.020917669, 0.020702716, 0.07410524, -0.014280978, 0.014469062, -0.002559291, 0.023953889, -0.005760084, -0.0067206575, 0.005615662, -0.008020455, -0.0023846412, -0.050594695, 0.0042889956, 0.029314293, 0.0110029355, 0.008007021, -0.016470818, -0.04911689, 0.033398412, -0.0093706325, 0.0066904..."
4,b'9fe1bb1f-4347-4298-8e44-d8fd7b204621',"b'The Galactic ISO Population 5\nThis particular form for the density profile has the advantage that the Poisson point process likelihood takes the\ntractable form\nlnL(logA , aR, az, \xcf\x840, \xcf\x89) = const + N\x10\nlogA\xe2\x88\x92aR\xe2\x9f\xa8R\xe2\x88\x92R0\xe2\x9f\xa9 \xe2\x88\x92az\x...","[0.027188687, 0.039433565, 0.049963366, 0.009532659, 0.027055735, -0.01587447, 0.02615166, -0.0009730425, 0.00030662058, -0.0009290022, -0.01235789, 0.01174631, -0.03773178, 0.009545954, 0.0138801895, 0.028903767, -0.031828713, -0.0153958425, -0.039353795, -0.019091908, 0.036083177, 0.007272475,..."


#### Set up the LlamaIndex Query Engine

In [19]:
query_engine = index.as_query_engine(
    similarity_top_k=5,
)

## 4. Retrieve Similar Sentences & RAG



Now that the embeddings are stored in KDB.AI, we can perform semantic search using through the LlamaIndex query engine.

### Search 1


In [22]:
#search_term1 = "The Milky Way is thought to host a huge population of interstellar objects (ISOs), numbering"
search_term1 = "what is the estimated population of interstellar objects (ISOs) in the milky way"

In [23]:
retrieved_chunks = query_engine.retrieve(search_term1)
for i in retrieved_chunks:
    print(i.node.get_text())
    print("____________________")

Draft version August 14, 2023
Typeset using L ATEX default style in AASTeX631
The Galactic Interstellar Object Population: A Framework for Prediction and Inference
Matthew J. Hopkins
 ,1Chris Lintott
 ,1Michele T. Bannister
 ,2J. Ted Mackereth
 ,3, 4, 5, ∗and
John C. Forbes
2
1Department of Physics, University of Oxford, Denys Wilkinson Building, Keble Road, Oxford, OX1 3RH, UK
2School of Physical and Chemical Sciences—Te Kura Mat¯ u, University of Canterbury, Private Bag 4800, Christchurch 8140, New Zealand
3Just Group plc, Enterprise House, Bancroft road, Reigate, Surrey RH2 7RP, UK
4Canadian Institute for Theoretical Astrophysics, University of Toronto, 60 St. George Street, Toronto, ON, M5S 3H8, Canada
5Dunlap Institute for Astronomy and Astrophysics, University of Toronto, 50 St. George Street, Toronto, ON M5S 3H4, Canada
ABSTRACT
The Milky Way is thought to host a huge population of interstellar objects (ISOs), numbering
approximately 1015pc−3around the Sun, which are formed and 

We can see these sentences do reference our search term 'number of interstellar objects in the milky way' in some way.

### Now we can perform RAG, passing the retrieved chunks from above to the LLM for a generate response:

In [24]:
result = query_engine.query(search_term1)
print(result.response)

The estimated population of interstellar objects (ISOs) in the Milky Way is approximately 10^15 per cubic parsec around the Sun.


### Search 2

Let's try another search term.

In [25]:
search_term2 = "how does planet formation occur"

In [26]:
retrieved_chunks = query_engine.retrieve(search_term2)
for i in retrieved_chunks:
    print(i.node.get_text())
    print("____________________")

14 Hopkins et al.
because the probability that two randomly selected ISOs have the same parent star is approximately equal to one over
the total number of stars which could contribute ISOs to the population around the Sun. The ISO sample likelihood
can then be combined with priors to form a posterior distribution on the model parameters:
p(θ|O1, . . . ,ON)∝p(θ)·Y
ip(Oi|θ) (10)
This posterior can be used to calculate estimates and confidence intervals for the values of the parameters θ. An
example of this calculation is given in section 6.3.
5.2. Additional Properties of ISOs
As with all minor planets, there are many observable quantities for ISOs that could be used in this framework in
the set of observable properties O. If the distribution of ISOs in these properties can be predicted, the inference
method of §5.1 can be used to compare the predicted distribution to the observed distribution in these properties, to
make inferences about the models used. If multiple properties are inclu

Again, we can see these sentences do reference our search term 'how does planet formation occur' in some way.

In [27]:
result = query_engine.query(search_term2)
print(result.response)

Planet formation occurs within protoplanetary disks, which are composed of gas and dust surrounding young stars. These disks provide the necessary material for the formation of planetesimals, which are the building blocks of planets. As dust particles collide and stick together, they form larger bodies, eventually leading to the creation of planetesimals. Over time, these planetesimals can collide and merge, forming larger objects that may become planets. The process is influenced by various factors, including the metallicity of the star, which affects the availability of materials necessary for planet formation. Additionally, dynamical interactions, such as scattering by giant planets or close stellar flybys, can eject some of these bodies into interstellar space, contributing to the population of interstellar objects.


## 5. Delete the KDB.AI Table

Once finished with the table, it is best practice to drop it.

In [28]:
table.drop()

True

## Take Our Survey

We hope you found this sample helpful! Your feedback is important to us, and we would appreciate it if you could take a moment to fill out our brief survey. Your input helps us improve our content.

[**Take the Survey**](https://delighted.com/t/vm88uwcB)